In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point, LineString
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.spatial import cKDTree
from shapely.ops import nearest_points

def calculate_route_distance(route_line, point1, point2):
    """
    Calculate the distance between two points on the bus route.
    
    Parameters:
    - route_line: LineString, representing the entire route
    - point1: Point, first point
    - point2: Point, second point
    
    Returns:
    - distance: float, the distance between the two points on the route
    """
    try:
        # Project points onto the route line
        projection1 = route_line.project(point1)
        projection2 = route_line.project(point2)
        
        # Ensure the projections are within bounds
        projection1 = max(0, min(projection1, route_line.length))
        projection2 = max(0, min(projection2, route_line.length))

        projected_point1 = route_line.interpolate(projection1)
        projected_point2 = route_line.interpolate(projection2)
        
        # Create a LineString between the two projected points
        segment = LineString([projected_point1, projected_point2])
        
        # Calculate the distance
        distance = segment.length
        return distance
    except Exception as e:
        print(f"Error calculating route distance: {e}, point1={point1}, point2={point2}")
        return None


In [ ]:
# Load the stop lines data
stop_lines = gpd.read_parquet('Data/stop_lines_cut.parquet')

# Ensure CRS matches, if not reproject
runs_crs = "EPSG:32632"  # Example CRS; replace with the correct one if different
if stop_lines.crs != runs_crs:
    stop_lines = stop_lines.to_crs(runs_crs)

# Example usage
stop_line_name = 'Goethestraße'
stop_line = stop_lines[stop_lines['Stop Name'] == stop_line_name].iloc[0]
stop_line_point = stop_line.geometry

# Initialize the list to store all data
all_data = []

file_paths = [
    'new_cleaned/cleaned_november.parquet',
    'new_cleaned/cleaned_december.parquet',
    'new_cleaned/cleaned_January.parquet',
    'new_cleaned/cleaned_october.parquet',
    'new_cleaned/cleaned_february.parquet',
    'new_cleaned/cleaned_march.parquet',
    'new_cleaned/cleaned_april.parquet'
]

# Load all data files
runs_list = [gpd.read_parquet(file_path) for file_path in file_paths]
runs = pd.concat(runs_list, ignore_index=True)

# Ensure CRS matches
if runs.crs != stop_lines.crs:
    runs = runs.to_crs(stop_lines.crs)

# Extract the first ten runs to create the route
first_two_runs_ids = runs['run'].unique()[:10]
first_two_runs = runs[runs['run'].isin(first_two_runs_ids)].sort_values(by=['run', 'utcTime'])

# Create a LineString object representing the entire route from the first two runs
route_points = first_two_runs.geometry.tolist()
route_line = LineString(route_points)

# Convert the LineString to a GeoDataFrame
route_line_gdf = gpd.GeoDataFrame(
    {'geometry': [route_line]},
    crs=runs.crs
)


In [ ]:
# Prepare the dataset for models
data = []

# Iterate through each run
for run_id, run in runs.groupby('run'):
    run = run.sort_values(by='utcTime').reset_index(drop=True)
    
    # Check if the run duration is less than 3 minutes
    run_start_time = run.loc[0, 'utcTime']
    run_end_time = run.loc[len(run) - 1, 'utcTime']
    run_duration = (run_end_time - run_start_time).total_seconds()
    if run_duration >= 2 * 60:
        continue  # Skip this run if duration is more than or equal to 3 minutes
    
    nearest_stop_point, _ = nearest_points(run.geometry.unary_union, stop_line_point)
    nearest_stop_idx = run.index[run.geometry == nearest_stop_point][0]
    nearest_stop_time = run.loc[nearest_stop_idx, 'utcTime']
    
    for i, row in run.iterrows():
        current_point = row.geometry
        current_time = row.utcTime
        
        if i >= nearest_stop_idx:
            continue
        
        distance = calculate_route_distance(route_line_gdf.geometry.iloc[0], current_point, stop_line_point)
        
        if distance is None:
            continue
        
        time_difference = (nearest_stop_time - current_time).total_seconds()
        if time_difference < 0:
            continue
        
  
        year_week = current_time.strftime('%Y-%U')
        
        data.append([run_id, distance, time_difference, year_week])
        print([run_id, distance, time_difference, year_week])

print(len(data))

# Convert the data to a DataFrame
df = pd.DataFrame(data, columns=['run', 'distance', 'time_difference', 'year_week'])

In [ ]:
# Bin the distances to integer values
df['distance_bin'] = (df['distance'] // 2) * 2

# Get unique weeks
unique_weeks = df['year_week'].unique()

# Split weeks into training and testing sets (80% training, 20% testing)
train_weeks, test_weeks = train_test_split(unique_weeks, test_size=0.2, random_state=42)

# Assign data to training and testing sets based on the week split
train_df = df[df['year_week'].isin(train_weeks)]
test_df = df[df['year_week'].isin(test_weeks)]

# Group by binned distance and calculate average time difference
grouped_train_data = train_df.groupby('distance_bin')['time_difference'].mean().reset_index()
grouped_train_data = grouped_train_data.rename(columns={'distance_bin': 'distance'})


# Create a KDTree for fast nearest neighbor search
kdtree = cKDTree(grouped_train_data[['distance']].values)

# Initialize lists to store actual and predicted values
actual_values = []
predicted_values = []

# Iterate through each point in the test set
for index, row in tqdm(test_df.iterrows(), total=test_df.shape[0], desc="Processing Test Points"):
    distance = row['distance']
    actual_time_difference = row['time_difference']
    
    # Find the closest distance in the grouped training data
    dist, idx = kdtree.query([[distance]], k=1)
    predicted_time_difference = grouped_train_data.iloc[idx[0]]['time_difference']
    
    # Append actual and predicted time differences to the lists
    actual_values.append(actual_time_difference)
    predicted_values.append(predicted_time_difference)

In [ ]:
# Convert to numpy arrays for RMSE and R-squared calculation
actual_values = np.array(actual_values)
predicted_values = np.array(predicted_values)

# Calculate RMSE and R-squared
rmse = np.sqrt(mean_squared_error(actual_values, predicted_values))
r2 = r2_score(actual_values, predicted_values)

# Print RMSE and R-squared
print(f'RMSE: {rmse}')
print(f'R-squared: {r2}')

# Plot actual vs. predicted values
plt.figure(figsize=(10, 6))
plt.scatter(actual_values, predicted_values, alpha=0.01)
plt.plot([actual_values.min(), actual_values.max()], [actual_values.min(), actual_values.max()], 'r--')
plt.xlabel('Actual Time Difference (seconds)')
plt.ylabel('Predicted Time Difference (seconds)')
plt.title('Actual vs Predicted Time Differences')
plt.show()

